# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
collection_names = db.list_collection_names()
print("Collections in 'uk_food' database:", collection_names)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6667539441a3cfccfbacd65d, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
documents = list(establishments.find(query))
df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print( len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Case-insensitive regex search for 'London'
    "RatingValue": {"$gte": 4}  # Greater than or equal to 4
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(count)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame

documents = list(establishments.find(query))
df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print( len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142  
longitude = 0.083840

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}
sort = [("scores.Hygiene", 1)]
limit = establishments.find(query).sort(sort).limit(10)


# Print the results
print(limit)

In [ ]:
# Convert result to Pandas DataFrame
top_establishments_list = list(limit)

# Convert the list of documents to a DataFrame
df = pd.DataFrame(top_establishments_list)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Step 1: Match documents where hygiene score is 0
    {"$group": {                        # Step 2: Group by Local Authority
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}            # Count the number of documents in each group
    }},
    {"$sort": {"count": -1}}            # Step 3: Sort the results by 'count' descending
]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
print( len(results))

# Print the first 10 results
for result in results[:10]: 
    print(result)

In [ ]:
# Convert the result to a Pandas DataFrame

df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))